<h2 style="text-align:center;font-size:200%;;"> CPS in Smart Home</h2>

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from matplotlib import pyplot as plt
import seaborn as sns
import os
import changefinder
from scipy import stats
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
import shap
shap.initjs()
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from tabulate import tabulate
from IPython.display import HTML, display

In [ ]:
df = pd.read_csv(r"C:\Users\Harsha\OneDrive\Desktop\DSA EL\New folder\modified.csv",low_memory=False)
df.columns = [i.replace(' [kW]', '') for i in df.columns]
print(f'HomeC.csv : {df.shape}')
df.head(3)

HomeC.csv : (39076, 35)


,month,datetime,use,gen,Home use[kW],Office consumption,Wine cellar,Garage door,kitchen,Barn,...,Furnace 1,Furnace 2,Smart_Lights_kW,Computer_kW,Printer_kW,AC_kW,Dishwasher,Fridge,other devices,Microwave
0,1,01-01-2016 05:00,0.209638,0.017417,2.349621,0.029230,0.003492,0.006542,0.064408,0.015675,...,0.010350,0.030958,0.001959,0.020868,0.007860,0.058962,0.000017,0.062075,0.000283,0.002033
1,1,01-01-2016 05:10,0.221461,0.017333,2.356371,0.032244,0.003492,0.006558,0.064316,0.015750,...,0.010358,0.031908,0.001908,0.019938,0.008944,0.067849,0.000000,0.062000,0.000283,0.002033
2,1,01-01-2016 05:20,0.221503,0.017333,2.355120,0.032610,0.003492,0.006542,0.064108,0.015758,...,0.010350,0.031158,0.005694,0.019387,0.008815,0.069629,0.000008,0.061767,0.000300,0.002033


<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

In [ ]:
df['use_HO'] = df['use']
df['gen_Sol'] = df['gen']
df.head(3)

,month,datetime,use,gen,Home use[kW],Office consumption,Wine cellar,Garage door,kitchen,Barn,...,Smart_Lights_kW,Computer_kW,Printer_kW,AC_kW,Dishwasher,Fridge,other devices,Microwave,use_HO,gen_Sol
0,1,01-01-2016 05:00,0.209638,0.017417,2.349621,0.029230,0.003492,0.006542,0.064408,0.015675,...,0.001959,0.020868,0.007860,0.058962,0.000017,0.062075,0.000283,0.002033,0.209638,0.017417
1,1,01-01-2016 05:10,0.221461,0.017333,2.356371,0.032244,0.003492,0.006558,0.064316,0.015750,...,0.001908,0.019938,0.008944,0.067849,0.000000,0.062000,0.000283,0.002033,0.221461,0.017333
2,1,01-01-2016 05:20,0.221503,0.017333,2.355120,0.032610,0.003492,0.006542,0.064108,0.015758,...,0.005694,0.019387,0.008815,0.069629,0.000008,0.061767,0.000300,0.002033,0.221503,0.017333


In [ ]:

# Apply range filter
df_filtered = df[(df['use_HO'] >= 0) & (df['use_HO'] <= 4) & (df['gen_Sol'] >= 0) & (df['gen_Sol'] <= 4)]
use = hv.Distribution(df_filtered['use_HO']).opts(title="Total Energy Consumption Distribution", color="red")
gen = hv.Distribution(df_filtered['gen_Sol']).opts(title="Total Energy Generation Distribution", color="blue")
(use + gen).opts(opts.Distribution(xlabel="Energy Consumption", ylabel="Density", xformatter='%.1fkWh', width=400, height=300, tools=['hover'], show_grid=True))


:Layout
   .Distribution.I  :Distribution   [use_HO]   (Density)
   .Distribution.II :Distribution   [gen_Sol]   (Density)

In [ ]:
print(df.columns)

Index(['month', 'datetime', 'use', 'gen', 'Home use[kW]', 'Office consumption',
       'Wine cellar', 'Garage door', 'kitchen', 'Barn', 'Well', 'living room',
       'Solar', 'temperature', 'humidity', 'visibility', 'apparentTemperature',
       'pressure', 'windSpeed', 'windBearing', 'precipIntensity', 'dewPoint',
       'precipProbability', 'Smart_TV_kW', 'Smart_AC_kW', 'Furnace 1',
       'Furnace 2', 'Smart_Lights_kW', 'Computer_kW', 'Printer_kW', 'AC_kW',
       'Dishwasher', 'Fridge', ' other devices', 'Microwave', 'use_HO',
       'gen_Sol'],
      dtype='object')


In [ ]:
import itertools
import holoviews as hv
colors = ["red", "blue", "orange", "green", "purple", "grey", "pink", "yellow", "brown", "skyblue", "lightgreen"]
color_cycle = itertools.cycle(colors)
distributions = []
exclude_columns = ['temperature', 'wine','month', 'datetime','Barn', 'Well','Solar', 'temperature', 'humidity', 'visibility', 'apparentTemperature','pressure', 'windSpeed', 'windBearing', 'precipIntensity', 'dewPoint','precipProbability','Unnamed: 35','use_HO', 'gen_Sol']
for col in df.columns[1:]:  # Exclude 'datetime'
    if df[col].dtype == 'float64' and col not in exclude_columns:  # Only consider numeric columns and not in exclude_columns
        color = next(color_cycle)
        distribution = hv.Distribution(df[(df[col] >= 0) & (df[col] <= 0.2)][col], label=col).opts(color=color, title=f'Energy Consumption Distribution of {col}')
        distributions.append(distribution)
layout = hv.Layout(distributions).opts(
    opts.Distribution(xlabel="Energy Consumption", ylabel="Density", xformatter='%.1f kW', width=800, height=350, tools=['hover'], show_grid=True)
)
layout


:Layout
   .Distribution.Use                                                  :Distribution   [use]   (Density)
   .Distribution.Gen                                                  :Distribution   [gen]   (Density)
   .Distribution.Home_use_left_square_bracket_kW_right_square_bracket :Distribution   [Home use[kW]]   (Density)
   .Distribution.Office_consumption                                   :Distribution   [Office consumption]   (Density)
   .Distribution.Wine_cellar                                          :Distribution   [Wine cellar]   (Density)
   .Distribution.Garage_door                                          :Distribution   [Garage door]   (Density)
   .Distribution.Kitchen                                              :Distribution   [kitchen]   (Density)
   .Distribution.Living_room                                          :Distribution   [living room]   (Density)
   .Distribution.Smart_TV_kW                                          :Distribution   [Smart_TV_kW]   (Density)
   .Distribution.Smart_AC_kW                                          :Distribution   [Smart_AC_kW]   (Density)
   .Distribution.Furnace_1                                            :Distribution   [Furnace 1]   (Density)
   .Distribution.Furnace_2                                            :Distribution   [Furnace 2]   (Density)
   .Distribution.Smart_Lights_kW                                      :Distribution   [Smart_Lights_kW]   (Density)
   .Distribution.Computer_kW                                          :Distribution   [Computer_kW]   (Density)
   .Distribution.Printer_kW                                           :Distribution   [Printer_kW]   (Density)
   .Distribution.AC_kW                                                :Distribution   [AC_kW]   (Density)
   .Distribution.Dishwasher                                           :Distribution   [Dishwasher]   (Density)
   .Distribution.Fridge                                               :Distribution   [Fridge]   (Density)
   .Distribution.Other_devices                                        :Distribution   [ other devices]   (Density)
   .Distribution.Microwave                                            :Distribution   [Microwave]   (Density)

### Weather Information

In [ ]:
temp = hv.Distribution(df['temperature'],label="temperature").opts(color="red")
apTemp = hv.Distribution(df['apparentTemperature'],label="apparentTemperature").opts(color="orange")
temps = (temp * apTemp).opts(opts.Distribution(title='Temperature Distribution')).opts(legend_position='top',legend_cols=2)
hmd = hv.Distribution(df['humidity']).opts(color="yellow", title='Humidity Distribution')
vis = hv.Distribution(df['visibility']).opts(color="blue", title='Visibility Distribution')
prs = hv.Distribution(df['pressure']).opts(color="green", title='Pressure Distribution')
wnd = hv.Distribution(df['windSpeed']).opts(color="purple", title='WindSpeed Distribution')
prc = hv.Distribution(df['precipIntensity']).opts(color="skyblue", title='PrecipIntensity Distribution')
dew = hv.Distribution(df['dewPoint']).opts(color="lightgreen", title='DewPoint Distribution')

(temps + hmd + vis + prs + wnd + prc + dew).opts(opts.Distribution(xlabel="Values", ylabel="Density", width=400, height=300,tools=['hover'],show_grid=True)).cols(4)

:Layout
   .Overlay.I        :Overlay
      .Distribution.Temperature         :Distribution   [temperature]   (Density)
      .Distribution.ApparentTemperature :Distribution   [apparentTemperature]   (Density)
   .Distribution.I   :Distribution   [humidity]   (Density)
   .Distribution.II  :Distribution   [visibility]   (Density)
   .Distribution.III :Distribution   [pressure]   (Density)
   .Distribution.IV  :Distribution   [windSpeed]   (Density)
   .Distribution.V   :Distribution   [precipIntensity]   (Density)
   .Distribution.VI  :Distribution   [dewPoint]   (Density)

In [ ]:
def groupByMonth(col):
    return df[[col,'month']].groupby('month').agg({col:['mean']})[col]

In [ ]:
def groupByWeekday(col):
    weekdayDf = df.groupby('weekday').agg({col:['mean']})
    weekdayDf.columns = [f"{i[0]}_{i[1]}" for i in weekdayDf.columns]
    weekdayDf['week_num'] = [['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'].index(i) for i in weekdayDf.index]
    weekdayDf.sort_values('week_num', inplace=True)
    weekdayDf.drop('week_num', axis=1, inplace=True)
    return weekdayDf

In [ ]:
def groupByTiming(col):
    timingDf = df.groupby('timing').agg({col:['mean']})
    timingDf.columns = [f"{i[0]}_{i[1]}" for i in timingDf.columns]
    timingDf['timing_num'] = [['Morning','Afternoon','Evening','Night'].index(i) for i in timingDf.index]
    timingDf.sort_values('timing_num', inplace=True)
    timingDf.drop('timing_num', axis=1, inplace=True)
    return timingDf

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'], format='%d-%m-%Y %H:%M')
var_df = df.resample('H').mean()

### LightGBM<a id='lgbm'></a>

In [ ]:
_lgbm_df = df.resample('H').mean()
_lgbm_df['weekday'] = LabelEncoder().fit_transform(_lgbm_df.index.day_name()).astype(np.int8)
_lgbm_df['hour'] = _lgbm_df.index.hour  # Extract hour from the datetime index
_lgbm_df['timing'] = LabelEncoder().fit_transform(_lgbm_df['hour']).astype(np.int8)


def lgbm_train(cols=['temperature','dewPoint','use_HO'],trg='use_HO',train_ratio=0.8,valid_ratio=0.1,test_ratio=0.1):
    #make dataframe for training
    lgbm_df = _lgbm_df[cols]
    tr,vd,te = [int(len(lgbm_df) * i) for i in [train_ratio, valid_ratio, test_ratio]]
    X_train, Y_train = lgbm_df[0:tr].drop([trg], axis=1), lgbm_df[0:tr][trg]
    X_valid, Y_valid = lgbm_df[tr:tr+vd].drop([trg], axis=1), lgbm_df[tr:tr+vd][trg]
    X_test = lgbm_df[tr+vd:tr+vd+te+2].drop([trg], axis=1)
    lgb_train = lgb.Dataset(X_train, Y_train)
    lgb_valid = lgb.Dataset(X_valid, Y_valid, reference=lgb_train)
    return lgb_train, lgb_valid, X_train, Y_train, X_valid, Y_valid


_lgbm_df['year'] = _lgbm_df.index.year
_lgbm_df['month'] = _lgbm_df.index.month
_lgbm_df['day'] = _lgbm_df.index.day
_lgbm_df['weekofyear'] = _lgbm_df.index.isocalendar().week

lgb_train, lgb_valid, X_train, Y_train, X_valid, Y_valid = lgbm_train(
    cols=['temperature', 'humidity', 'visibility', 'apparentTemperature',
          'pressure', 'windSpeed', 'windBearing', 'precipIntensity',
          'dewPoint', 'precipProbability', 'year', 'month', 'day', 'weekday', 'weekofyear',
          'hour', 'timing', 'use_HO'],
    trg='use_HO', train_ratio=0.9, valid_ratio=0.09, test_ratio=0.01)

params = {
    'task': 'train',
    'boosting': 'gbdt',
    'objective': 'regression',
    'metric': 'l2',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0,
    'min_child_samples': 5,
}

# Model training
gbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=[lgb_train, lgb_valid])



In [ ]:
def lgbm_train(cols=['temperature','dewPoint','use_HO'],trg='use_HO',train_ratio=0.8,valid_ratio=0.1,test_ratio=0.1):
    #make dataframe for training
    lgbm_df = _lgbm_df[cols]
    tr,vd,te = [int(len(lgbm_df) * i) for i in [train_ratio, valid_ratio, test_ratio]]
    X_train, Y_train = lgbm_df[0:tr].drop([trg], axis=1), lgbm_df[0:tr][trg]
    X_valid, Y_valid = lgbm_df[tr:tr+vd].drop([trg], axis=1), lgbm_df[tr:tr+vd][trg]
    X_test = lgbm_df[tr+vd:tr+vd+te+2].drop([trg], axis=1)
    lgb_train = lgb.Dataset(X_train, Y_train)
    lgb_valid = lgb.Dataset(X_valid, Y_valid, reference=lgb_train)
    #model training
    params = {
        'task' : 'train',
        'boosting':'gbdt',
        'objective' : 'regression',
        'metric' : {'mse'},
        'num_leaves':200,
        'drop_rate':0.05,
        'learning_rate':0.1,
        'seed':0,
        'feature_fraction':1.0,
        'bagging_fraction':1.0,
        'bagging_freq':0,
        'min_child_samples':5
    }
    gbm = lgb.train(params, lgb_train, num_boost_round=100, valid_sets=[lgb_train, lgb_valid])
    #make predict dataframe
    pre_df = pd.DataFrame()
    pre_df[trg] = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    pre_df.index = lgbm_df.index[tr+vd:tr+vd+te+2]
    return pre_df, gbm, X_train
lgbmForecast_df, model, x_train = lgbm_train(\
                cols=['temperature', 'humidity', 'visibility', 'apparentTemperature',\
                       'pressure', 'windSpeed', 'windBearing', 'precipIntensity',\
                       'dewPoint', 'precipProbability','year', 'month','day', 'weekday', 'weekofyear', \
                        'hour', 'timing','use_HO'],\
                trg='use_HO',train_ratio=0.9,valid_ratio=0.09,test_ratio=0.01)

In [ ]:
lgbmForecast_df, model, x_train = lgbm_train(\
                cols=['temperature', 'humidity', 'visibility', 'apparentTemperature',\
                       'pressure', 'windSpeed', 'windBearing', 'precipIntensity',\
                       'dewPoint', 'precipProbability','year', 'month','day', 'weekday', 'weekofyear', \
                        'hour', 'timing','use_HO'],\
                trg='use_HO',train_ratio=0.9,valid_ratio=0.09,test_ratio=0.01)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000937 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2006
[LightGBM] [Info] Number of data points in the train set: 5861, number of used features: 16
[LightGBM] [Info] Start training from score 0.455308


In [ ]:
    #evaluation with MAE
    lgbm_use_mae = mean_absolute_error(_lgbm_df['use_HO'][-len(lgbmForecast_df):], lgbmForecast_df['use_HO'])
    ((hv.Curve(_lgbm_df['use_HO'], label='use_HO').opts(color='blue')\
    * hv.Curve(lgbmForecast_df['use_HO'], label='use_HO predicted').opts(color='red', title='LightGBM Result - Total Energy Consumption')).opts(legend_position='bottom') + \
    (hv.Curve(_lgbm_df['use_HO'][-int(len(_lgbm_df)*0.05):], label='use_HO').opts(color='blue') \
    * hv.Curve(lgbmForecast_df['use_HO'], label='use_HO predicted').opts(color='red', title='LightGBM Result Enlarged - Total Energy Consumption')).opts(legend_position='bottom'))\
        .opts(opts.Curve(xlabel="Time", yformatter='%.2fkw', width=800, height=300, show_grid=True, tools=['hover'])).opts(shared_axes=False).cols(1)

:Layout
   .Overlay.I  :Overlay
      .Curve.Use_HO           :Curve   [datetime]   (use_HO)
      .Curve.Use_HO_predicted :Curve   [datetime]   (use_HO)
   .Overlay.II :Overlay
      .Curve.Use_HO           :Curve   [datetime]   (use_HO)
      .Curve.Use_HO_predicted :Curve   [datetime]   (use_HO)

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

In [ ]:
def grangerTestPlot(weather_info, applicances, _maxlag):
    grangerTest_df = pd.DataFrame()
    for weather in weather_info:
        for appliance in applicances:
            test_result = grangercausalitytests(df[[appliance, weather]], maxlag=_maxlag, verbose=False)
            p_values = [round(test_result[i][0]['ssr_chi2test'][1],4) for i in range(1, _maxlag+1)]
            min_p_value = np.min(p_values)
            grangerTest_df.loc[appliance, weather] = min_p_value

    fig,ax = plt.subplots(figsize=(10, 8)) 
    sns.heatmap(grangerTest_df, vmax=1, vmin=0, center=1, annot=True)
    ax.set_title('Granger Causality Test Result',size=20)
    plt.xlabel("Weather Information",size=15)
    plt.ylabel("Energy Consumption",size=15)
    plt.show()

In [ ]:
for col in ['temperature', 'humidity', 'windSpeed', 'windBearing', 'precipIntensity', 'dewPoint', 'use_HO', 'Wine cellar']:

    resampled_col = df[col].resample('H').mean()
    if resampled_col.var() > 0:  # Check if variance is greater than 0
        p_value = adfuller(resampled_col, autolag='AIC', regression='ct')[1]
        print(f"p-value for {col}: {p_value}")
    else:
        print(f"Skipping {col} because it is constant")


In [ ]:
var_df = df.resample('H').mean()
def var_train(cols=['temperature', 'humidity', 'visibility',  'windSpeed', 'windBearing', 'dewPoint','Furnace', 'use_HO'], max_order=10, train_ratio=0.9,test_ratio=0.1):


    tr,te = [int(len(var_df) * i) for i in [train_ratio, test_ratio]]
    train, test = var_df[0:tr], var_df[tr:]


    var_func = VAR(train[cols], freq='H')
    var_func.select_order(max_order)
    model = var_func.fit(maxlags=max_order, ic='aic', trend='ct')
    model_result = model.summary()

    varForecast_df = pd.DataFrame(model.forecast(model.endog, steps=len(test)),columns=cols)
    varForecast_df.index = test.index
    
    return varForecast_df, model_result

In [ ]:
varForecast_df, model_result = var_train(cols=['temperature', 'humidity','windSpeed', 'windBearing', 'precipIntensity','dewPoint','use_HO','Wine cellar'], \
                                         max_order=48, train_ratio=0.99,test_ratio=0.01)

var_use_mae = mean_absolute_error(var_df['use_HO'][-len(varForecast_df):], varForecast_df['use_HO'])
((hv.Curve(var_df['use_HO'], label='use_HO').opts(color='blue')\
  * hv.Curve(varForecast_df['use_HO'], label='use_HO predicted').opts(color='red', title='VAR Result - Total Energy Consumption')).opts(legend_position='bottom') + \
 (hv.Curve(var_df['use_HO'][-int(len(var_df)*0.05):], label='use_HO').opts(color='blue') \
  * hv.Curve(varForecast_df['use_HO'], label='use_HO predicted').opts(color='red', title='VAR Result Enlarged - Total Energy Consumption')).opts(legend_position='bottom'))\
    .opts(opts.Curve(xlabel="Time", yformatter='%.2fkw', width=800, height=300, show_grid=True, tools=['hover'])).opts(shared_axes=False).cols(1)

:Layout
   .Overlay.I  :Overlay
      .Curve.Use_HO           :Curve   [datetime]   (use_HO)
      .Curve.Use_HO_predicted :Curve   [datetime]   (use_HO)
   .Overlay.II :Overlay
      .Curve.Use_HO           :Curve   [datetime]   (use_HO)
      .Curve.Use_HO_predicted :Curve   [datetime]   (use_HO)

In [ ]:
plots = []
devices = ["use",
"gen",
"Home use[kW]",
"Office consumption",
"Wine cellar",
"Garage door",
"kitchen",
"living room",
"Smart_AC_kW",
"Smart_Lights_kW",
"Computer_kW",
"Printer_kW",
"Dishwasher",
"Fridge",
"Microwave"]
for device in devices:
    lgbmForecast_df, model, x_train = lgbm_train(
        cols=[device, 'temperature', 'humidity', 'visibility', 'apparentTemperature',
              'pressure', 'windSpeed', 'windBearing', 'precipIntensity',
              'dewPoint', 'precipProbability', 'year', 'month', 'day', 'weekday', 'weekofyear',
              'hour', 'timing'],
        trg=device, train_ratio=0.9, valid_ratio=0.09, test_ratio=0.01)

    curve_full = hv.Curve(_lgbm_df[device], label=device).opts(color='blue', width=1500, height=400)
    curve_forecast = hv.Curve(lgbmForecast_df[device], label=device + ' predicted').opts(color='red', title=f'LightGBM Result - {device}', width=1500, height=400)
    plot_full = curve_full * curve_forecast
    plots.append(plot_full)

layout = hv.Layout(plots).cols(1)
layout


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001582 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2006
[LightGBM] [Info] Number of data points in the train set: 5861, number of used features: 16
[LightGBM] [Info] Start training from score 0.455308
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001003 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2006
[LightGBM] [Info] Number of data points in the train set: 5861, number of used features: 16
[LightGBM] [Info] Start training from score 0.281863
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000898 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2006
[LightGBM] [Info] Number of data points in the train set: 5861, number of used features: 16
[LightGBM] [Info] Start traini

:Layout
   .Overlay.I    :Overlay
      .Curve.Use           :Curve   [datetime]   (use)
      .Curve.Use_predicted :Curve   [datetime]   (use)
   .Overlay.II   :Overlay
      .Curve.Gen           :Curve   [datetime]   (gen)
      .Curve.Gen_predicted :Curve   [datetime]   (gen)
   .Overlay.III  :Overlay
      .Curve.Home_use_left_square_bracket_kW_right_square_bracket           :Curve   [datetime]   (Home use[kW])
      .Curve.Home_use_left_square_bracket_kW_right_square_bracket_predicted :Curve   [datetime]   (Home use[kW])
   .Overlay.IV   :Overlay
      .Curve.Office_consumption           :Curve   [datetime]   (Office consumption)
      .Curve.Office_consumption_predicted :Curve   [datetime]   (Office consumption)
   .Overlay.V    :Overlay
      .Curve.Wine_cellar           :Curve   [datetime]   (Wine cellar)
      .Curve.Wine_cellar_predicted :Curve   [datetime]   (Wine cellar)
   .Overlay.VI   :Overlay
      .Curve.Garage_door           :Curve   [datetime]   (Garage door)
      .Curve.Garage_door_predicted :Curve   [datetime]   (Garage door)
   .Overlay.VII  :Overlay
      .Curve.Kitchen           :Curve   [datetime]   (kitchen)
      .Curve.Kitchen_predicted :Curve   [datetime]   (kitchen)
   .Overlay.VIII :Overlay
      .Curve.Living_room           :Curve   [datetime]   (living room)
      .Curve.Living_room_predicted :Curve   [datetime]   (living room)
   .Overlay.IX   :Overlay
      .Curve.Smart_AC_kW           :Curve   [datetime]   (Smart_AC_kW)
      .Curve.Smart_AC_kW_predicted :Curve   [datetime]   (Smart_AC_kW)
   .Overlay.X    :Overlay
      .Curve.Smart_Lights_kW           :Curve   [datetime]   (Smart_Lights_kW)
      .Curve.Smart_Lights_kW_predicted :Curve   [datetime]   (Smart_Lights_kW)
   .Overlay.XI   :Overlay
      .Curve.Computer_kW           :Curve   [datetime]   (Computer_kW)
      .Curve.Computer_kW_predicted :Curve   [datetime]   (Computer_kW)
   .Overlay.XII  :Overlay
      .Curve.Printer_kW           :Curve   [datetime]   (Printer_kW)
      .Curve.Printer_kW_predicted :Curve   [datetime]   (Printer_kW)
   .Overlay.XIII :Overlay
      .Curve.Dishwasher           :Curve   [datetime]   (Dishwasher)
      .Curve.Dishwasher_predicted :Curve   [datetime]   (Dishwasher)
   .Overlay.XIV  :Overlay
      .Curve.Fridge           :Curve   [datetime]   (Fridge)
      .Curve.Fridge_predicted :Curve   [datetime]   (Fridge)
   .Overlay.XV   :Overlay
      .Curve.Microwave           :Curve   [datetime]   (Microwave)
      .Curve.Microwave_predicted :Curve   [datetime]   (Microwave)